In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler


import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import acquire
import wrangle

In [2]:
df = acquire.get_telco_data()

In [3]:
df = wrangle.wrangle_telco()

In [4]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,streaming_movies,paperless_billing,contract_type_id,payment_type_id,internet_service_type_id,churn,automatic_payment,total_charges_scaled,tenure_less_than_12,tenure_less_than_7
0,0002-ORFBO,Female,0,Yes,Yes,9,65.6,593.30,Yes,No,...,No,Yes,2,2,1,No,False,0.066294,True,False
1,0003-MKNFE,Male,0,No,No,9,59.9,542.40,Yes,Yes,...,Yes,No,1,2,1,No,False,0.060420,True,False
2,0004-TLHLJ,Male,0,No,No,4,73.9,280.85,Yes,No,...,No,Yes,1,1,2,Yes,False,0.030239,True,True
3,0011-IGKFF,Male,1,Yes,No,13,98.0,1237.85,Yes,No,...,Yes,Yes,1,1,2,Yes,False,0.140670,False,False
4,0013-EXCHZ,Female,1,Yes,No,3,83.9,267.40,Yes,No,...,No,Yes,1,2,2,Yes,False,0.028687,True,True


In [5]:
train, test = train_test_split(df, random_state=123, train_size=.80)

In [6]:
encoder = LabelEncoder()

encode_list = [
    'gender', 'partner', 'dependents', 'phone_service'
    , 'multiple_lines', 'online_security', 'online_backup'
    , 'device_protection', 'tech_support'
    , 'streaming_movies', 'streaming_tv', 'paperless_billing', 'churn'
    ]
               
               
               
for e in encode_list:
    train[e] = encoder.fit_transform(train[e])
    test[e] = encoder.transform(test[e])

In [7]:
train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,streaming_movies,paperless_billing,contract_type_id,payment_type_id,internet_service_type_id,churn,automatic_payment,total_charges_scaled,tenure_less_than_12,tenure_less_than_7
6924,9824-BEMCV,1,0,1,1,17,69.00,1149.65,1,0,...,0,1,1,1,2,1,False,0.130493,False,False
5249,7398-LXGYX,1,0,1,0,44,84.80,3626.35,1,2,...,0,1,1,4,2,0,True,0.416288,False,False
5852,8225-BTJAU,1,1,0,0,10,79.25,793.55,1,0,...,2,1,1,1,2,1,False,0.089401,True,False
3597,5099-BAILX,1,1,1,1,43,110.75,4687.90,1,2,...,2,1,1,3,2,1,True,0.538784,False,False
4147,5862-BRIXZ,1,0,0,0,46,60.75,2893.40,0,1,...,2,1,3,3,1,0,True,0.331710,False,False


# Logit classifier without total_charges_scaled

In [8]:
# train, validate = train_test_split(train, random_state=123, test_size=.2)


# X_train = train[['senior_citizen', 'multiple_lines', 'streaming_tv', 'paperless_billing', 'contract_type_id', 'automatic_payment']]
# y_train = train[['churn']]
# X_val = validate[['senior_citizen', 'multiple_lines', 'streaming_tv', 'paperless_billing', 'contract_type_id', 'automatic_payment']]
# y_val = validate[['churn']]
# X_test = test[['senior_citizen', 'multiple_lines', 'streaming_tv', 'paperless_billing', 'contract_type_id', 'automatic_payment']]
# y_test = test[['churn']]

In [9]:
# logit = LogisticRegression(random_state = 123)
# logit.fit(X_train, y_train)

In [10]:
# y_pred = logit.predict(X_train)
# y_pred[:10]

In [11]:
# print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
#      .format(logit.score(X_train, y_train)))

In [12]:
# pd.DataFrame(confusion_matrix(y_train, y_pred))

In [13]:
# model_df = train[['churn']]
# model_df.rename(columns={'churn': 'actual'})

In [14]:
# model_df['logit_predictions'] = y_pred
# model_df.head()

In [15]:
# print(classification_report(y_train, y_pred))

# Logit classifier with tenure less than 7

In [16]:
train, validate = train_test_split(train, random_state=123, test_size=.2)


X_train = train[['contract_type_id', 'automatic_payment', 'total_charges_scaled', 'tenure_less_than_7']]
y_train = train[['churn']]
X_val = validate[['contract_type_id', 'automatic_payment', 'total_charges_scaled', 'tenure_less_than_7']]
y_val = validate[['churn']]
X_test = test[['contract_type_id', 'automatic_payment', 'total_charges_scaled', 'tenure_less_than_7']]
y_test = test[['churn']]

In [17]:
logit = LogisticRegression(random_state = 123)
logit.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [18]:
y_pred = logit.predict(X_train)
y_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [19]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set: 0.75


In [20]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.90      0.84      3316
           1       0.56      0.36      0.44      1191

    accuracy                           0.75      4507
   macro avg       0.68      0.63      0.64      4507
weighted avg       0.73      0.75      0.74      4507



# Logit classifier with tenure less than 12

In [21]:
X_train1 = train[['contract_type_id', 'automatic_payment', 'total_charges_scaled', 'tenure_less_than_12']]
y_train1 = train[['churn']]
X_val1 = validate[['contract_type_id', 'automatic_payment', 'total_charges_scaled', 'tenure_less_than_12']]
y_val1 = validate[['churn']]
X_test1 = test[['contract_type_id', 'automatic_payment', 'total_charges_scaled', 'tenure_less_than_12']]
y_test1 = test[['churn']]

In [22]:
logit.fit(X_train1, y_train1)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
y_pred1 = logit.predict(X_train1)
y_pred1[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [24]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train1, y_train1)))

Accuracy of Logistic Regression classifier on training set: 0.75


In [25]:
print(classification_report(y_train1, y_pred1))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83      3316
           1       0.53      0.45      0.48      1191

    accuracy                           0.75      4507
   macro avg       0.67      0.65      0.66      4507
weighted avg       0.74      0.75      0.74      4507



# Logit classifier with ss

In [32]:
X_train2 = train[['senior_citizen', 'contract_type_id', 'automatic_payment', 'tenure_less_than_7']]
y_train2 = train[['churn']]
X_val2 = validate[['senior_citizen','contract_type_id', 'automatic_payment', 'tenure_less_than_7']]
y_val2 = validate[['churn']]
X_test2 = test[['senior_citizen', 'contract_type_id', 'automatic_payment', 'tenure_less_than_7']]
y_test2 = test[['churn']]

In [33]:
logit.fit(X_train2, y_train2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [34]:
y_pred2 = logit.predict(X_train2)
y_pred2[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [35]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train2, y_train2)))

Accuracy of Logistic Regression classifier on training set: 0.76


In [36]:
print(classification_report(y_train1, y_pred1))

              precision    recall  f1-score   support

           0       0.81      0.85      0.83      3316
           1       0.53      0.45      0.48      1191

    accuracy                           0.75      4507
   macro avg       0.67      0.65      0.66      4507
weighted avg       0.74      0.75      0.74      4507

